# A0149963M
## Assignment 4
## YSC2229 - Introductory Data Structures and Algorithms

### Solution for Problem 1 - Runway Scheduling System

**1(a) - Store an approved/valid request in O(log n), where the stored information includes: aircraft ID (with format "AA123456") and take-off time**

Before starting on this question, I'd like to define the following basic objects first. Firstly, i define the constants Red and Black which constitute the basis of Red-Black Trees. Secondly, I define the class RBNode which represents a Node in the Red-Black Tree. This stores the following attributes: the time (in 24h format), its ID number, its parent (self.p), left and right child, colour (red by default) and info, which will be covered later when we talk about augmenting the Red-Black Tree in order to output requests in **O(log n)**. In addition, I will store the time object, which is needed to store the airplane times. 

In [34]:
Red = 'Red'
Black = 'Black'

class RBNode():
    def __init__(self, id_no = None, time = None, info = None):
        self.time = time
        self.id_no = id_no
        self.p = None
        self.left = None
        self.right = None
        self.col = Red 
        self.info = info

class Time():
    def __init__(self, h, minute, sec):
        """
        Stores time in hours, minutes
        and seconds. 
        """
        self.check_validity(h, minute, sec)
        self.h = h
        self.minute = minute
        self.sec = sec 
    
    def check_validity(self, h, minute, sec):
        """
        Utility function to check if time inputted
        is a valid time to be registered. 
        """
        if type(h) != int or type(minute) != int or type(sec) != int:
            raise TypeError("Please input Integer values only!")
        
        if h < 0 or h > 23:
            raise ValueError("Please Input integer hour value between 0 and 23 inclusive.")
        
        if minute < 0 or minute > 59:
            raise ValueError("Please Input integer minute value between 0 and 59 inclusive.")
    
        if sec < 0 or sec > 59:
            raise ValueError("Please Input integer second value between 0 and 59 inclusive.")
    
    def in_sec(self):
        """
        Converts the time into seconds. This helps to directly compare 
        2 times and see which one is later and which is earlier. 
        """
        n_sec = self.h * 3600 + self.minute * 60 + self.sec
        return n_sec
    
    def fix_time(self, unit):
        """
        Utility function to help express time in terms of hh-mm-ss format
        """
        if unit < 10:
            return "0" + str(unit)
        else:
            return str(unit)
    
    def in_str(self):
        """
        Pretty print to express term in hh-mm-ss format 
        """
        str_time = self.fix_time(self.h) + ":" + self.fix_time(self.minute) + ":" + self.fix_time(self.sec)
        return str_time
    
    

Next, I will be doing brief tests of the Time object and demonstrate what it is supposed to do that I have defined above. In the next isolated square, I have also tested the **check_validity** auxillary function that I have defined for time and created some illegal expressions that I have commented out. feel free to uncomment it if you'd like to test it.

In [36]:
# testing time initialization
# input in hours, then minutes, then seconds
t = Time(22, 34, 10)

# gives 81250 seconds, which is correct
t.in_sec()

# pretty print format
print("24h format time of Time(22, 34, 10):" ,t.in_str())

# Now demonstrating fix_time for units < 10
t = Time(8, 5, 3)
print("24h format time of Time(8, 5, 3):" ,t.in_str())

24h format time of Time(22, 34, 10): 22:34:10
24h format time of Time(8, 5, 3): 08:05:03


In [45]:
# t = Time("", 3, 1)
# t = Time(2, 2.3, 1)
# t = Time(25, 0, 1)
# t = Time(23, -1, 3)
# t = Time(20, 1, 65)

With this set in stone, it is time to define the Red-Black Tree proper. The code of this follows the lecture notes very closely, and it contains all necessary operations such as **left_rotate**, **right_rotate**, **fix_insert**, **insert**, etc. However, for this question, it is mentioned that a request is stored/approved if is is not within **k** minutes from the previous and the next requests that are already stored in the system. 

Therefore, we have to define a few additional functions in order to address this requirement. In particular, there must be an auxillary function that checks the parent, left and right node to verify whether it is within **k** minutes from the previous and the next requests that are already stored in the system. In addition, we need to define a new attribute delay that becomes the basis of comparison when checking for this requirement. 

Other than that, the Red Black Tree has been calibrated in order to search for Time With these requirements set in stone, let's go ahead and create the Red-Black Tree data structure:

In [51]:
class RedBlackTree():
    def __init__(self, delay):
        """
        NIL node is defined as per normal, but
        delay is converted from minutes to seconds.
        This makes for easy comparison with the 
        in_sec operation defined earlier in Time().
        """
        self.delay = 60 * delay
        self.NIL = RBNode()
        self.NIL.col = Black
        self.NIL.left = None
        self.NIL.right = None
        self.root = self.NIL
    
    def left_rotate(self, x):
        """
        Conventional rotation operation taken
        from the module textbook
        """
        y = x.right
        x.right = y.left
        if y.left != self.NIL:
            y.left.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.left:
            x.p.left = y
        else:
            x.p.right = y
        y.left = x
        x.p = y
    
    def right_rotate(self, x):
        """
        Same as left_rotate, just mirrored
        """
        y = x.left
        x.left = y.right
        if y.right != self.NIL:
            y.right.p = x
        
        y.p = x.p
        if x.p == self.NIL:
            self.root = y
        elif x == x.p.right:
            x.p.right = y
        else: 
            x.p.left = y
        y.right = x
        x.p = y
    
        
    def fix_insert(self, z):
        """
        Adjustment procedure to maintain
        Red-Black property! 
        """
        while z.p.col == Red:
            if z.p == z.p.p.left:
                y = z.p.p.right
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.right:
                        z = z.p
                        self.left_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.right_rotate(z.p.p)
            else:
                y = z.p.p.left
                if y.col == Red:
                    z.p.col = Black
                    y.col = Black
                    z.p.p.col = Red
                    z = z.p.p
                else:
                    if z == z.p.left:
                        z = z.p
                        self.right_rotate(z)
                    z.p.col = Black
                    z.p.p.col = Red
                    self.left_rotate(z.p.p)
                    
        self.root.col = Black
      
        
    def insert(self, id_key, hh_key, mm_key, ss_key):
        """
        Insert operation. The only modification here
        is that instead of storing an integer, a Time
        object is stored instead. 
        """
        time_key = Time(hh_key, mm_key, ss_key)
        z = RBNode(id_key, time_key)
        y = self.NIL
        x = self.root
        
        while x != self.NIL:
            y = x
            if z.time.in_sec() < x.time.in_sec():
                x = x.left
            else: 
                x = x.right
            
        z.p = y
        if y == self.NIL:
            self.root = z
        elif z.time.in_sec() < y.time.in_sec():
            y.left = z
        else: 
            y.right = z
        
        z.left = self.NIL
        z.right = self.NIL
        z.col = Red
        
        self.fix_insert(z)
        
        
    def find_min(self):
        """
        Function to find minimum value of Red-Black Tree
        by traversing all the way to the left-most node. 
        """
        node = self.root
        while node.left != self.NIL:
            node = node.left
        print("Minimum time is:", node.time.in_str())
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def find_max(self):
        """
        Function to find maximum value of Red-Black Tree
        by traversing all the way to the left-most node. 
        """
        node = self.root
        while node.right != self.NIL:
            node = node.right
        print("Maximum time is:", node.time.in_str())
        print("Corresponding ID number is:", node.id_no, "\n")
    
    def is_valid(self):
        """
        Checks the neighbour of the node it is about to
        be inserted to on whether it is eligible to be
        approved or not. 
        """
        pass
    
    def print_node(self, x):
        """
        Auxillary function to pretty print node values. 
        """
        print("ID:", x.id_no, ", Time:", x.time.in_str())
    
    def inorder(self, node):
        """
        In order traversal of the node that 
        also prints values on the way.
        """
        if node != self.NIL:
            self.inorder(node.left)
            self.print_node(node)
            self.inorder(node.right)
            
    def populate_info(self, node):
        """
        Populates the node with information about
        its sub-tree. Very useful for the search
        operation later on.
        """
        if node is None or node.time == None:
            return []
        left = self.populate_info(node.left)
        right = self.populate_info(node.right)
        node.info = left + [(node.id_no, node.time.in_str())] + right
        return node.info
    

When inserting data into this Red-Black Tree, please take note on how to add the node. The insert function takes 4 arguments, in this order: **tree.insert(ID_No, hh, mm, ss)**. Subsequently, **hh, mm, ss** is converted to a time object and added to the tree. Some examples on how to do so are shown below, together with informative print messages:

In [50]:
t = RedBlackTree(3) # min delay is 3min
t.insert("AR190237", 13, 34, 59) # 13:34:59
t.insert("AS123984", 21, 12, 0)  # 21:12:00
t.insert("AE123973", 19, 7, 12)  # 19:07:00 
t.insert("AR812731", 15, 12, 9)  # 15:12:09
t.insert("AS239183", 8, 3, 2)    # 21:12:00

t.find_min()
t.find_max()

t.inorder(t.root)
t.populate_info(t.root)

Minimum time is: 08:03:02
Corresponding ID number is: AS239183 

Maximum time is: 21:12:00
Corresponding ID number is: AS123984 

ID: AS239183 , Time: 08:03:02
ID: AR190237 , Time: 13:34:59
ID: AR812731 , Time: 15:12:09
ID: AE123973 , Time: 19:07:12
ID: AS123984 , Time: 21:12:00


[('AS239183', '08:03:02'),
 ('AR190237', '13:34:59'),
 ('AR812731', '15:12:09'),
 ('AE123973', '19:07:12'),
 ('AS123984', '21:12:00')]

Now that we are done with the basic functions, let's test our binary tree implementation. I have 3 functions here, that each do a different thing:
 - rand_tree creates a random binary tree for testing. The randomness helps ensure robustness
 - check_RB checks whether the Red-Black property is violated in random trees (Node red -> child nodes black)
 - check_RB_path checks whether the Red-Black property is satisfied in terms of 
 - check_height checks whether the Red-Black true has a height of roughly **log(n)**, where n is the number of nodes

In [ ]:
def rand_tree():
    """
    Generates a random Red-Black tree of
    a maximum number of nodes (due to delay
    restriction) and IDs randomized to fit 
    the question. Delay set to 0 to ensure
    number of nodes added is deterministic. 
    """
    
    pass

def check_RB(t):
    """
    Helper function to check whether Red-
    Black tree fits the requirement that
    no 2 adjacent nodes are red. 
    """
    pass

def check_RB_path(t):
    pass

def height(t):
    pass

def check_height(t):
    pass

Now that we are done with testing the function, 

### Solution for Problem 2 - Belief Propogation

For this question, I have to solve the belief propogation problem. In this problem, I have to solve the Belief Propogation Problem using Dynamic programming. I start from the first part:

**(a) Argue or justify that the problem can be solved using dynamic programming.**

This problem can be solved using dynamic programming. This is because of the overlapping substructures of the subproblems. In other words, subproblems share subsubproblems. If dynamic programming is not used, then the algorithm becomes inefficient because we have to compute the same subsubproblem multiple times. However, dynamic programing allows us to solve these subsubproblems just once and store its answer in a table, which can be queried, making our algorithm much more efficient. 

For a concrete example, consider the undirected graph consisting of the following:
 - Vertices consist of 1, 2, 3
 - Edge consists of (1, 2) and (1, 2)
Say, you'd like to calculate the eventual beliefs of all the vertices in this graph and you start from Node a. Following the definition defined in the assignment brief, then you have to calculate the following:

$$
b(x_{1} = T) = \phi(x_{1} = T) \times m_{12}(x_{1} = T) 
$$
$$
b(x_{1} = F) = \phi(x_{1} = F) \times m_{12}(x_{1} = F)
$$

Where $m_{12}$ represents the message from Vertice 2 to Vertice 1 (order swapped a little from the assignment brief). However, we also note that 

In [55]:
import random

class Influence:
    
    def __init__(self, coef = None):
        """
        XOR for F-T, T-F between nodes,
        AND for F-F, T-T between nodes.
        """
        self.coef = coef
        self.AND = random.random()
        self.XOR = 1 - self.AND
        self.set_param()
        
    def set_param(self):
        """
        Auxillary function to 
        """
        if self.coef and (self.coef >= 0 and self.coef <= 1):
            self.XOR = self.coef
            self.AND = 1 - self.coef
        elif not self.coef:
            return
        else:
            print("Please input a probability between 0 and 1. Randomized XOR, AND used instead")
            return

class Node:
    def __init__(self, name, coef = None):
        self.coef = coef 
        self.name = name
        self.T = random.random()
        self.F = 1 - self.T
        self.label = None # not sure whether needed 
        self.link = {}
        self.set_param()
        
    def set_param(self):
        if self.coef and (self.coef >= 0 and self.coef <= 1):
            self.T = self.coef
            self.F = 1 - self.coef
        elif not self.coef:
            return
        else:
            print("Please input a probability between 0 and 1. Randomized T and F used instead")
            return

Next, we define a new class called Graph here. In it, 

In [53]:
class Graph:
    def __init__(self):
        self.G = {}
        self.memo = {}
    
    def add_node(self, name, coef = None):
        """
        """
        n = Node(name, coef)
        if n.name in self.G:
            print("Node already in Graph. Aborting... ")
            return
        else:
            self.G[n.name] = n
            return
    
    def add_edge(self, n1, n2, coef = None):
        """
        """
        if n1 in self.G and n2 in self.G:
            i = Influence(coef)
            self.G[n1].link[n2] = i
            self.G[n2].link[n1] = i
        else:
            print("2 nodes not found in graph. Aborting... ")
            return 
            
    def print_graph(self):
        """
        """
        for node in self.G:
            print("Now looking at Node: ", self.G[node].name) 
            print("T:", self.G[node].T, ", F:", self.G[node].F)
            print("Neighbouring edges: ")
            for edge, val in self.G[node].link.items():
                print("Connection between Node", node, "and", edge + ":")
                print("XOR val:", val.XOR) 
                print("AND val:", val.AND, "\n")
            print("\n")
            

In [54]:
q = Graph()
q.add_node("a", 0.7)
q.add_node("b", 0.4)
q.add_node("c", 0.3)
q.add_node("d", 0.2)
q.add_edge("a", "b", 0.8)
q.add_edge("b", "c", 0.4)
q.add_edge("c", "d", 0.5)
q.print_graph()

NameError: name 'Node' is not defined

In [52]:
def message(t, s, b, graph):
    """
    t - target
    s - source
    n - neighbour
    b - belief
    """
    if b == "T":
        val = s.link[t.name].AND * s.T + s.link[t.name].XOR * s.F
        for n in s.link:
            if n != t.name:
                val *= message(s, graph.G[n], "T", graph)
                val *= message(s, graph.G[n], "F", graph) 
    else:
        val = s.link[t.name].XOR * s.T + s.link[t.name].AND * s.F
        for n in s.link:
            if n != t.name:
                val *= message(s, graph.G[n], "F", graph)
                val *= message(s, graph.G[n], "T", graph)
    return val 


def find_belief(graph):
    for (label, node) in graph.G.items():
        b_true = node.T
        b_false = node.F
        for (key, _) in node.link.items():
            source = graph.G[key]
            b_true *= message(node, source, "T", graph)
            b_false *= message(node, source, "F", graph)
        
        print(b_true, b_false)
        node.label = b_true > b_false
    return graph
find_belief(q)
# q.G["a"].label

NameError: name 'q' is not defined

Solution for Problem 3 - Huffman Code

In [13]:
class HuffmanNode:
    def __init__(self, freq, char = None, left = None, right = None):
        self.left = left
        self.right = right
        self.char = char
        self.freq = freq
        self.table = {}

In [12]:
import sys

class MinPriorityQueue:
    def __init__(self, capacity):
        self.limit = capacity 
        self.size = 0
        self.Heap = [None] * (capacity + 1)
        self.Heap[0] = HuffmanNode(sys.maxsize * -1)
        self.root = 1

    
    def swap(self, pos1, pos2):
        self.Heap[pos1], self.Heap[pos2] = self.Heap[pos2], self.Heap[pos1]
        
    def parent(self, pos):
        return pos // 2
    
    def left(self, pos):
        return pos * 2
    
    def right(self, pos):
        return pos * 2 + 1
    
    def min_heapify(self, pos):
        l = self.left(pos)
        r = self.right(pos)
        if l <= self.size and self.Heap[l].freq < self.Heap[pos].freq:
            low = l
        else:
            low = pos
        if r <= self.size and self.Heap[r].freq < self.Heap[low].freq:
            low = r
        if low != pos:
            self.swap(pos, low)
            self.min_heapify(low)
        
    def build_minheap(self):
        for i in range(self.size // 2, 0, -1):
            self.min_heapify(i)
            
    def heap_min(self):
        return self.Heap[1]
            
    def extract_min(self):
        if self.size < 1:
            raise ValueError("Heap Underflow")
        low = self.Heap[1]
        self.Heap[1] = self.Heap[self.size]
        self.Heap[self.size] = None
        self.size -= 1
        self.min_heapify(1)
        return low
    
    def decrease_key(self,i, key):
        if key.freq > self.Heap[i].freq:
            raise ValueError("New key Larger than Current key")
        self.Heap[i] = key
        while i > 1 and self.Heap[self.parent(i)].freq > self.Heap[i].freq:
            self.swap(self.parent(i), i)
            i = self.parent(i)
            
    def insert(self,key):
        self.size += 1
        self.Heap[self.size] = HuffmanNode(sys.maxsize)
        self.decrease_key(self.size, key)
    
   

In [18]:
def char_count(s):
    seen = {}
    for c in s:
        if c not in seen:
            seen[c] = 1
        else:
            seen[c] += 1
    return seen

def assign_code(node, root, seq = ""):
    if type(node.char) == str:
        root.table[node.char] = seq                
    else:                              
        assign_code(node.left, root, seq + "0")    
        assign_code(node.right, root, seq + "1")
    
def encode(sentence, huffman):
    out = ""
    for char in sentence:
        out = out + huffman.table[char]
    
    return out
    
def huffman_code(arr):
    """
    arr - array of sentences
    """
    for sentence in arr:
        
        table = char_count(sentence)
        q = MinPriorityQueue(len(table.keys()))
        
        for char, freq in table.items():
            q.insert(HuffmanNode(freq, char))
        
        for i in range(len(table.keys()) - 1):
            x = q.extract_min()
            y = q.extract_min()
            z = HuffmanNode(x.freq + y.freq, None, x, y)
            q.insert(z)
        
        huffman = q.extract_min()
        assign_code(huffman, huffman)
        print(encode(sentence, huffman))
        print(q.Heap)
    
        
        

In [19]:
huffman_code(["999999999ssssssss888772"])

0000000001111111111111111100100100101110111010
[<__main__.HuffmanNode object at 0x000001A845927788>, None, None, None, None, None]


Solution for Problem 4 - Maximum Flow/ Minimum Cut Problem